In [1]:
import pickle
import baseline_nel
import knowledge_graph_extraction
import pandas as pd
from tqdm import tqdm

In [9]:
with open('../data/sample_data_candidates.pkl', 'rb') as f:
    data = pickle.load(f)

In [10]:
len(data)

4978

In [11]:
# db = "../data/kensho.db"

# query = """
# SELECT 
#     source_item_id, edge_property_id, target_item_id
# FROM 
#     wikipage_triplets
# """

# triplets = knowledge_graph_extraction.conduct_sql_query(db, query)

In [12]:
# baseline_nel.predict_local(triplets, data[0][0], data[0][1], top_k)

In [13]:
predictions = []
triplets_dicts = []
top_k = 10
try:
    for i in tqdm(range(len(data))):
        if i % 500 == 0:
            with open('../data/sample_baseline_predictions_' + str(i) + '.pkl', 'wb') as f:
                pickle.dump(predictions, f)
            with open('../data/sample_baseline_triplets_' + str(i) + '.pkl', 'wb') as f:
                pickle.dump(triplets_dicts, f)
        row = data[i]
        prediction, triplets = baseline_nel.predict("../data/kensho.db", "wikipage_triplets", row[0], row[1], top_k)
        predictions.append(prediction)
        triplets_dicts.append(triplets)
except:
    with open('../data/error_' + str(i), 'wb') as f:
        pickle.dump(['error at '+ str(i)], f)

100%|██████████| 4978/4978 [4:37:22<00:00,  2.30s/it]  


In [ ]:
baseline_nel.predict("../data/kensho.db", "wikipage_triplets", row[0], row[1], top_k)

In [14]:
with open('../data/sample_baseline_predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)
with open('../data/sample_baseline_triplets.pkl', 'wb') as f:
    pickle.dump(triplets_dicts, f)

In [15]:
with open('../data/sample_baseline_predictions.pkl', 'rb') as f:
    predictions = pickle.load(f)

In [16]:
predictions_dict = {prediction[0] : prediction[1] for prediction in predictions}
candidates_dict = {candidate[0]: candidate[1] for candidate in data}

In [17]:
def compute_accuracy(ground_truth_dict, prediction_dict, top_k):
    total = 0
    correct = 0
    for k, v in ground_truth_dict.items():
        if top_k is None:
            if k in prediction_dict and v in prediction_dict[k]:
                correct += 1
        else:
            if k in prediction_dict and v in prediction_dict[k][:top_k]:
                correct += 1
        total += 1
    if total == 0:
        return 0, 0
    return correct, total

In [2]:
with open('../data/sample_labels.pkl', 'rb') as f:
    labels = pickle.load(f)

In [3]:
labels[0]

['Llewellyn Heycock, Baron Heycock Order of the British Empire (12 August 1905 &ndash; 13 March 1990) was a Wales local politician, who became a Life peer in 1967. Heycock was born in Margam and began his career as an engine driver with the Great Western Railway. He subsequently rose to a powerful position in South Wales local politics through his Trade union connections and membership of the Labour Party, a "personality of transcendent authority". Despite having himself received little formal education, he became Chairman of the Glamorganshire Education Committee. In April 1967 he was elected as a county councillor to Glamorgan County Council for the Port Talbot East ward. In 1973 he was elected unopposed as councillor for Margam Central on the new West Glamorgan. He became a Order of the British Empire (CBE) in 1959, a Commander of the Order of Saint John (CStJ) in April 1967, and a life peer on 10 July 1967 as Baron Heycock, of Taibach in the Borough of Port Talbot.',
 {('Order of t

In [19]:
len(labels)

4978

In [20]:
for top_k in [1, 5, 10]:
    total_correct, total_num_labels = 0, 0
    for i, row in enumerate(labels):
        text = row[0]
        prediction= predictions_dict[text]
        prediction = {key[:3]: value for key, value in prediction.items()}
        label = {key[:3]: key[3] for key in row[1].keys()}
        correct, num_labels = compute_accuracy(label, prediction, top_k)
        total_correct += correct
        total_num_labels += num_labels 
    print("Accuracy for Top {}: {}".format(top_k, total_correct / total_num_labels))
    

Accuracy for Top 1: 0.23113695551353858
Accuracy for Top 5: 0.3344511433769755
Accuracy for Top 10: 0.3735150369234062


In [21]:
total_correct, total_num_labels = 0, 0
for i, row in enumerate(labels):
    text = row[0]
    candidate= candidates_dict[text]
    candidate = {key[:3]: value for key, value in candidate.items()}
    label = {key[:3]: key[3] for key in row[1].keys()}
    correct, num_labels = compute_accuracy(label, candidate, None)
    total_correct += correct
    total_num_labels += num_labels 
print("Percent of Correct Wikidata item in Candidates list: {}".format(total_correct / total_num_labels))
    

Percent of Correct Wikidata item in Candidates list: 0.5431486568442082
